In [1]:
#Max Shi CS559 Homework 4 Q2
import math
import numpy as np

In [2]:
def Gaussian(X, mu, sigma):
    D = X.shape[1]
    sigdet = np.linalg.det(sigma)
    sigma_inv = np.linalg.inv(sigma)
    result = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        X_mu = X[i] - mu
        X_mu_T = X_mu.T
        RHS = np.exp(-0.5 * (X_mu_T @ sigma_inv @ X_mu))
        LHS = 1 / (2 * math.pi) ** (D / 2) / math.sqrt(sigdet)
        result[i] = LHS * RHS
    return result

In [3]:
def Expectation(X, k, pi, mu, sigma):
    n = X.shape[0]
    weights = np.zeros((n, k))
    for k_ind in range(k):
        weights[:, k_ind] = pi[k_ind] * Gaussian(X, mu[k_ind, :], sigma[k_ind, :, :])
    weights = weights / np.sum(weights, axis=1, keepdims=True)
    return weights


In [4]:
def MaximizeMean(X, k, r):
    n = X.shape[0]
    D = X.shape[1]
    mu = np.zeros((k, D))
    for k_ind in range(k):
        mu[k_ind, :] = np.sum(np.transpose([r[:, k_ind]])* X, axis=0) / np.sum(r[:, k_ind])
    return mu


In [5]:
def MaximizeCovariance(X, k, r, mu):
    n = X.shape[0]
    D = X.shape[1]
    sigma = np.zeros((D, D, k))
    for k_ind in range(k):
        for n_ind in range(n):
            X_mu = np.array([X[n_ind] - mu[k_ind, :]])
            sigma[k_ind, :, :] += r[n_ind, k_ind] * X_mu.T @ X_mu
        sigma[k_ind, :, :] /= np.sum(r[:, k_ind])
    return sigma

In [6]:
def MaximizeMixtures(k,r):
    pi = np.sum(r, axis=0) / np.sum(r)
    return pi

In [7]:
def LogLikelihood(X, pi, mu, sigma):
    n = X.shape[0]
    k = pi.shape[0]
    result = 0
    for i in range(n):
        for k_ind in range(k):
            currentX = np.array([X[i]])
            result += math.log(pi[k_ind] * Gaussian(currentX, mu[k_ind, :], sigma[k_ind, :, :]))
    return result

In [8]:
def EM(X, K, pi0, mu0, sigma0, niter):
    n = X.shape[0]
    D = X.shape[1]
    k = K
    pi = pi0
    mu = mu0
    sigma = sigma0
    r = np.zeros((n, k, k))
    for i in range(niter):
        r = Expectation(X, k, pi, mu, sigma)
        pi = MaximizeMixtures(k, r)
        mu = MaximizeMean(X, k, r)
        sigma = MaximizeCovariance(X, k, r, mu)
        print('pi', pi)
        print("log likelihood of iteration %d: %d " % (i, LogLikelihood(X, pi, mu, sigma)))
    return pi, mu, sigma, r

In [9]:
file = np.genfromtxt("points.dat.txt")
X = file
pi0 = np.array([0.5, 0.5])
mu0 = np.array([[0, 0], [1, 1]])
sigma0 = np.array([[[1, 0.5], [0.5, 1]], [[1, 0.5], [0.5, 1]]])
niter = 12

pi, mu, sigma, r = EM(X, 2, pi0, mu0, sigma0, niter)
print(pi, mu, sigma, r)

pi [0.67689083 0.32310917]
log likelihood of iteration 0: -13228 
pi [0.6635645 0.3364355]
log likelihood of iteration 1: -14116 
pi [0.65490605 0.34509395]
log likelihood of iteration 2: -14928 
pi [0.65020678 0.34979322]
log likelihood of iteration 3: -15544 
pi [0.64990584 0.35009416]
log likelihood of iteration 4: -16037 
pi [0.65181676 0.34818324]
log likelihood of iteration 5: -16902 
pi [0.65100978 0.34899022]
log likelihood of iteration 6: -18100 
pi [0.64994428 0.35005572]
log likelihood of iteration 7: -18649 
pi [0.64973909 0.35026091]
log likelihood of iteration 8: -18799 
pi [0.6495026 0.3504974]
log likelihood of iteration 9: -18824 
pi [0.64908873 0.35091127]
log likelihood of iteration 10: -18815 
pi [0.64858222 0.35141778]
log likelihood of iteration 11: -18798 
[0.64858222 0.35141778] [[-1.17924435 -1.91154464]
 [-0.15819271  1.87127671]] [[[11.20894086 -1.72347679]
  [-1.72347679  4.04269871]]

 [[ 1.18767561  0.3755709 ]
  [ 0.3755709   1.01652594]]] [[9.99999110e-0